## EXERCISE 1

<img src='https://drive.google.com/uc?id=1C2vyT2X19C_IekWf3rELuF4SumvJxo2b'>

1) Write a Python script that computes P(g|do(r)) seen during the lecture

2) What is the causal effect of the cloudy weather $c$ on the grass' wetness $g$? (i.e. math expression of $P(g | do(c))$

3) Between rain and sprinkler, which one has the stronger causal effect on the wetness, $P(g | do(r))$ or $P(g | do(s))$ ?

4) Compute the average causal effect (ACE) of the sprinkler on the wetness $g$.

### 1. Write a Python script that computes P(g|do(r)) seen during the lecture

In [2]:
from IPython.display import display, Math, Latex

import pyAgrum as gum
import pyAgrum.lib.notebook as gnb
import pyAgrum.causal as csl
import pyAgrum.causal.notebook as cslnb

In [3]:
m1 = gum.fastBN("Cloudy{T|F}->Rain{T|F}->GrassWet{T|F};Cloudy{T|F}->Sprinkler{T|F}->GrassWet{T|F}")

m1.cpt("Cloudy")[:]=[0.5,0.5]
m1.cpt("Rain")[:]=[[0.8,0.2], [0.1,0.9]]
m1.cpt("Sprinkler")[:]=[[0.1,0.9], [0.5,0.5]]

m1.cpt("GrassWet")[{'Rain':'T','Sprinkler':'T'}] = [0.95,0.05]
m1.cpt("GrassWet")[{'Rain':'T','Sprinkler':'F'}] = [0.90,0.10]
m1.cpt("GrassWet")[{'Rain':'F','Sprinkler':'T'}] = [0.90,0.10]
m1.cpt("GrassWet")[{'Rain':'F','Sprinkler':'F'}] = [0.10,0.90]

gnb.flow.row(m1, m1.cpt("Cloudy"), m1.cpt("Rain"), m1.cpt("Sprinkler"), m1.cpt("GrassWet"))

In [12]:
d1 = csl.CausalModel(m1)
#cslnb.showCausalModel(d1)

d1.names()

{0: 'Cloudy', 1: 'Rain', 2: 'GrassWet', 3: 'Sprinkler'}

In [5]:
cslnb.showCausalImpact(d1, 'GrassWet', doing='Rain', values={'Rain':'T'})

In [6]:
ci1 = csl.causalImpact(d1, 'GrassWet', doing='Rain', values={'Rain':'T'})

g_do_r = ci1[1].toarray()
print(g_do_r)

[0.915 0.085]


### 2. What is the causal effect of the cloudy weather $c$ on the grass' wetness $g$? (i.e. math expression of $P(g | do(c))$

$Cloudy$ has no parents, so:
$$ P(g|do(c)) = P(g|c) $$

In [7]:
ci2 = csl.causalImpact(d1, 'GrassWet', doing='Cloudy', values={'Cloudy':'T'})

g_do_c = ci2[1].toarray()
print(g_do_c)

[0.76 0.24]


$$ P(g|do(c)) = P(g|c) = \frac{P(g,c)}{P(c)} = \alpha \sum_{r,s \in R,S} P(g,r,s,c) = \alpha \sum_{r,s \in R,S} P(c) P(r|c) P(s|c) P(g|r,s) \\ = \alpha P(c) \sum_{r \in R} P(r|c) \sum_{s \in S} P(s|c) P(g|r,s) = \alpha' \sum_{r \in R} P(r|c) \sum_{s \in S} P(s|c) P(g|r,s)$$
(c=T)

In [8]:
import numpy as np

print("Computing P(g|c) with the Bayesian network:")

t, f = 0, 1
P_C = np.array([0.5, 0.5])
P_S_C = np.array([[0.1, 0.5],[0.9, 0.5]])
P_R_C = np.array([[0.8, 0.1],[0.2, 0.9]])
P_G_SR = np.array([[[0.95, 0.9],[0.9, 0.1]],[[0.05, 0.1],[0.1, 0.9]]])

p = np.zeros(2)
for r in range(2):
    for s in range(2):
        p += P_R_C[r,t] * P_S_C[s,t] * P_G_SR[:,r,s]

a = np.sum(p)
p /= a
print(p)

Computing P(g|c) with the Bayesian network:
[0.76 0.24]


### 3. Between rain and sprinkler, which one has the stronger causal effect on the wetness, $P(g | do(r))$ or $P(g | do(s))$ ?

In [9]:
cslnb.showCausalImpact(d1, 'GrassWet', doing='Sprinkler', values={'Sprinkler':'T'})

In [10]:
ci3 = csl.causalImpact(d1, 'GrassWet', doing='Sprinkler', values={'Sprinkler':'T'})

g_do_s = ci3[1].toarray()
print(g_do_s)

if (g_do_s[0] > g_do_r[0]): print("Sprinkler wins!")
else: print("Rain wins!")

[0.9225 0.0775]
Sprinkler wins!


### 4. Compute the average causal effect (ACE) of the sprinkler on the wetness $g$.

In [11]:
s_t = g_do_s[0]
print("P(g|do(s)) =", s_t)

ci4 = csl.causalImpact(d1, 'GrassWet', doing='Sprinkler', values={'Sprinkler':'F'})

g_do_s2 = ci4[1].toarray()
s_f = g_do_s2[0]
print("P(g|do(not s)) =", s_f)

ace = s_t - s_f
print("ACE =", ace)

P(g|do(s)) = 0.9225
P(g|do(not s)) = 0.46
ACE = 0.46249999999999997
